In [2]:
import pyscf
import slowquant.SlowQuant as sq
from qiskit_nature.second_q.mappers import JordanWignerMapper
from slowquant.qiskit_interface.interface import QuantumInterface
from slowquant.qiskit_interface.wavefunction import WaveFunction
from slowquant.unitary_coupled_cluster.ups_wavefunction import WaveFunctionUPS
#from slowquant.qiskit_interface.linear_response.projected import quantumLR

from qiskit_ibm_runtime.fake_provider import FakeTorino

from qiskit.primitives import Estimator

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_aer.primitives import Sampler
from qiskit_aer.primitives import SamplerV2

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

import numpy as np
from scipy.optimize import minimize
import itertools
import random
from scipy.optimize import curve_fit
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
import heapq

In [11]:
mol = pyscf.M(atom="""H 0.0 0.0 0.0;
                      H 1.5 0.0 0.0;
                      H 0.0 1.8 0.0;
                      H 1.5 1.8 0.0;""", basis="sto-3g", unit="angstrom")
rhf = pyscf.scf.RHF(mol).run()

sampler = Estimator()
primitive = sampler
mapper = JordanWignerMapper()
# For H4 you can make the wavefunction better by increasing n_layers.
# n_layers: 3 will prob. give almost the FCI solution.
QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 1, "do_pp": True}, ISA = True)

WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (4, 4),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    #include_active_kappa = True
)

WF.run_vqe_2step("RotoSolve", maxiter=3)
WF.run_vqe_2step("SLSQP", maxiter=40)
#WF.ansatz_parameters = [3.142800380570141, 2.802123553483023, 3.141844220036336, 3.1415016877195234, 2.7699109323169533, 3.140289037489865, 1.5722892785591813, -0.05183267369041262, -3.139930460730367, -3.05193091891377, 0.07499512879186974, 0.26977250393898994, -2.875485939584386, -0.18478422435886988, 0.21835082743355141, -3.145279642647945, 0.2027868009455131, 3.141049426792193, -2.802097303067954, -0.20245039509840054, 0.14415193008627167, 0.24779481988227897, -0.04867618752527981, -3.0111535837495227, -3.1367777510378136, -0.21605050555915123, -3.14472880076259]
optimized_parameters = WF.ansatz_parameters
#no noise
nonclif_ground_state_energy = WF.energy_elec
print("Non-Clifford Ground state energy:", nonclif_ground_state_energy)

#device noise
# Update the primitive with simulated noise
backend = FakeTorino()
QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
QI.do_postselection = False
QI.do_M_mitigation = False
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(backend_options={"noise_model":noise_model})
WF.change_primitive(sampler)    
# Calculate the ground state energy using the noisy simulator
noisy_nonclif_ground_state_energy = WF.energy_elec
print("Noisy Non-Clifford Ground state energy:",noisy_nonclif_ground_state_energy)

converged SCF energy = -1.77674731624872


Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3
Full optimization
Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------Ansatz optimization
--------Iteration # | Iteration time [s] | Electronic energy [Hartree]


/tmp/ipykernel_669176/3974532165.py:7: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  sampler = Estimator()


--------     1      |        16.79       |     -3.5534864802975159    
--------     2      |        16.73       |     -3.5829207758700985    
--------     3      |        16.70       |     -3.5844408552283911    
Full optimization
Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------Ansatz optimization
--------Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------     1      |         6.22       |     -3.5845371988525310    
--------     2      |         6.11       |     -3.5845686529219343    
--------     3      |         6.16       |     -3.5845806681498735    
--------     4      |         6.12       |     -3.5845809600373078    
--------     5      |         6.11       |     -3.5845809668916440    
Non-Clifford Ground state energy: -3.584580966891644
Using this function is only recommended for switching from ideal simulator to shot-noise or quantum hardware.
                 Multiple switching back and forth can lead to un-expected outcomes and i

In [3]:
print(WF.ansatz_parameters)

[3.1415926535865144, -0.16027725679287372, 3.141592653585989, 3.1415926535934684, -0.17940404049528347, 3.1415926535940724, 1.5707962290294233, -0.16028326544864677, 3.141592504818808]


In [4]:
for i in range(len(optimized_parameters)):
    m = np.pi/2
    WF.ansatz_parameters[i] = m
    for n in range(9):
        if i == 1 + 3*n:
            WF.ansatz_parameters[i] = 4*m

print(WF.ansatz_parameters)

[1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966]


In [12]:
mol = pyscf.M(atom="""H 0.0 0.0 0.0;
                      H 1.5 0.0 0.0;
                      H 0.0 1.8 0.0;
                      H 1.5 1.8 0.0;""", basis="sto-3g", unit="angstrom")
rhf = pyscf.scf.RHF(mol).run()

sampler = Estimator()
primitive = sampler
mapper = JordanWignerMapper()
# For H4 you can make the wavefunction better by increasing n_layers.
# n_layers: 3 will prob. give almost the FCI solution.
QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 1, "do_pp": True}, ISA = True)

WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (4, 4),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    #include_active_kappa = True
)
WF.ansatz_parameters = [1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966]
#WF.ansatz_parameters = [np.pi/2 for _ in range(9)]
clifford_ground_state_energy = WF.energy_elec
print(clifford_ground_state_energy)

backend = FakeTorino()
QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
QI.do_postselection = False
QI.do_M_mitigation = False
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(backend_options={"noise_model":noise_model})
WF.change_primitive(sampler)
noisy_clifford_ground_state_energy = WF.energy_elec
print('exact:', clifford_ground_state_energy)
print('noisy:', noisy_clifford_ground_state_energy)

converged SCF energy = -1.77674731624872


Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3
-3.289581357882598


/tmp/ipykernel_669176/2572658892.py:7: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  sampler = Estimator()


Using this function is only recommended for switching from ideal simulator to shot-noise or quantum hardware.
                 Multiple switching back and forth can lead to un-expected outcomes and is an experimental feature.

Reset RDMs, energies, and QI metrics.
Number of shots is None. Ideal simulator is assumed.
Your settings are:
 Ansatz:              tUPS
 Number of shots:     None
 ISA                  True
 Transpiled circuit   True
 Primitive:           Sampler
 Post-processing:     False
 Circuit layout:      [17, 25, 35, 24, 28, 26, 27, 44]
 Non-local gates:     135
 Transpilation strategy: External PassManager
 Primitive backend:   AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['cz', 'x', 'id', 'reset', 'measure', 'sx']>)

exact: -3.289581357882598
noisy: -3.1485218507061687


In [13]:
mol = pyscf.M(atom="""H 0.0 0.0 0.0;
                      H 1.5 0.0 0.0;
                      H 0.0 1.8 0.0;
                      H 1.5 1.8 0.0;""", basis="sto-3g", unit="angstrom")
rhf = pyscf.scf.RHF(mol).run()

sampler = Estimator()
primitive = sampler
mapper = JordanWignerMapper()
# For H4 you can make the wavefunction better by increasing n_layers.
# n_layers: 3 will prob. give almost the FCI solution.
QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 2, "do_pp": True}, ISA = True)

WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (4, 4),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    #include_active_kappa = True
)

#WF.run_vqe_2step("RotoSolve", maxiter=3)
#WF.run_vqe_2step("SLSQP", maxiter=40)
WF.ansatz_parameters = [3.149292243857527, 2.7959119978771727, 3.149309991378343, -3.149408938522492, -0.34778596661917566, 3.1457594688862214, -1.6618954100368926, 0.05158209484284317, 3.238589976193641, -2.7513559843586957, -0.08687242149566897, 3.5388840968045137, 2.8029677052395936, -0.024123584231435992, 2.695058906228867, -3.163427097555806, -0.15200969007831952, 3.158390745058286]
optimized_parameters = WF.ansatz_parameters
#no noise
nonclif_ground_state_energy = WF.energy_elec
print("Non-Clifford Ground state energy:", nonclif_ground_state_energy)

#device noise
# Update the primitive with simulated noise
backend = FakeTorino()
QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
QI.do_postselection = False
QI.do_M_mitigation = False
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(backend_options={"noise_model":noise_model})
WF.change_primitive(sampler)    
# Calculate the ground state energy using the noisy simulator
noisy_nonclif_ground_state_energy = WF.energy_elec
print("Noisy Non-Clifford Ground state energy:",noisy_nonclif_ground_state_energy)

converged SCF energy = -1.77674731624872
Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3
Full optimization
Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------Ansatz optimization
--------Iteration # | Iteration time [s] | Electronic energy [Hartree]


/tmp/ipykernel_669176/3233408176.py:7: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  sampler = Estimator()


--------     1      |        34.51       |     -3.5535111635354757    
--------     2      |        34.99       |     -3.5832909529165948    
--------     3      |        34.81       |     -3.5859795034945612    
Full optimization
Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------Ansatz optimization
--------Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------     1      |        13.38       |     -3.5863447190015556    
--------     2      |        13.26       |     -3.5865664095879590    
--------     3      |        13.26       |     -3.5866533573327861    
--------     4      |        13.26       |     -3.5868952955885405    
--------     5      |        13.25       |     -3.5872882850862267    
--------     6      |        13.23       |     -3.5876506083163373    
--------     7      |        13.27       |     -3.5877101877749880    
--------     8      |        13.26       |     -3.5877227982728654    
--------     9      |        13.31     

In [15]:
print(len(WF.ansatz_parameters))

18


In [16]:
for i in range(len(optimized_parameters)):
    m = np.pi/2
    WF.ansatz_parameters[i] = m
    for n in range(9):
        if i == 1 + 3*n:
            WF.ansatz_parameters[i] = 4*m

print(WF.ansatz_parameters)

[1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966]


In [4]:
mol = pyscf.M(atom="""H 0.0 0.0 0.0;
                      H 1.5 0.0 0.0;
                      H 0.0 1.8 0.0;
                      H 1.5 1.8 0.0;""", basis="sto-3g", unit="angstrom")
rhf = pyscf.scf.RHF(mol).run()

sampler = Estimator()
primitive = sampler
mapper = JordanWignerMapper()
# For H4 you can make the wavefunction better by increasing n_layers.
# n_layers: 3 will prob. give almost the FCI solution.
QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 2, "do_pp": True}, ISA = True)

WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (4, 4),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    #include_active_kappa = True
)
#WF.ansatz_parameters = [1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966]
WF.ansatz_parameters = [np.pi/2 for _ in range(18)]
clifford_ground_state_energy = WF.energy_elec
print(clifford_ground_state_energy)

backend = FakeTorino()
QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
QI.do_postselection = False
QI.do_M_mitigation = False
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(backend_options={"noise_model":noise_model})
WF.change_primitive(sampler)
noisy_clifford_ground_state_energy = WF.energy_elec
print('exact:', clifford_ground_state_energy)
print('noisy:', noisy_clifford_ground_state_energy)

converged SCF energy = -1.77674731624872


Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3


/tmp/ipykernel_776274/622673437.py:7: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  sampler = Estimator()


-2.649219076334765
Using this function is only recommended for switching from ideal simulator to shot-noise or quantum hardware.
                 Multiple switching back and forth can lead to un-expected outcomes and is an experimental feature.

Reset RDMs, energies, and QI metrics.
Number of shots is None. Ideal simulator is assumed.
Your settings are:
 Ansatz:              tUPS
 Number of shots:     None
 ISA                  True
 Transpiled circuit   True
 Primitive:           Sampler
 Post-processing:     False
 Circuit layout:      [17, 25, 35, 24, 28, 26, 27, 44]
 Non-local gates:     270
 Transpilation strategy: External PassManager
 Primitive backend:   AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['measure', 'id', 'cz', 'sx', 'x', 'reset']>)

exact: -2.649219076334765
noisy: -2.573071328282566


In [19]:
mol = pyscf.M(atom="""H 0.0 0.0 0.0;
                      H 1.5 0.0 0.0;
                      H 0.0 1.8 0.0;
                      H 1.5 1.8 0.0;""", basis="sto-3g", unit="angstrom")
rhf = pyscf.scf.RHF(mol).run()

sampler = Estimator()
primitive = sampler
mapper = JordanWignerMapper()
# For H4 you can make the wavefunction better by increasing n_layers.
# n_layers: 3 will prob. give almost the FCI solution.
QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 3, "do_pp": True}, ISA = True)

WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (4, 4),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    include_active_kappa = True
)

#WF.run_vqe_2step("RotoSolve", maxiter=3)
#WF.run_vqe_2step("SLSQP", maxiter=40)
WF.ansatz_parameters = [3.142800380570141, 2.802123553483023, 3.141844220036336, 3.1415016877195234, 2.7699109323169533, 3.140289037489865, 1.5722892785591813, -0.05183267369041262, -3.139930460730367, -3.05193091891377, 0.07499512879186974, 0.26977250393898994, -2.875485939584386, -0.18478422435886988, 0.21835082743355141, -3.145279642647945, 0.2027868009455131, 3.141049426792193, -2.802097303067954, -0.20245039509840054, 0.14415193008627167, 0.24779481988227897, -0.04867618752527981, -3.0111535837495227, -3.1367777510378136, -0.21605050555915123, -3.14472880076259]
optimized_parameters = WF.ansatz_parameters
#no noise
nonclif_ground_state_energy = WF.energy_elec
print("Non-Clifford Ground state energy:", nonclif_ground_state_energy)

#device noise
# Update the primitive with simulated noise
backend = FakeTorino()
QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
QI.do_postselection = False
QI.do_M_mitigation = False
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(backend_options={"noise_model":noise_model})
WF.change_primitive(sampler)    
# Calculate the ground state energy using the noisy simulator
noisy_nonclif_ground_state_energy = WF.energy_elec
print("Noisy Non-Clifford Ground state energy:",noisy_nonclif_ground_state_energy)

converged SCF energy = -1.77674731624872


Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3


/tmp/ipykernel_669176/3359661477.py:7: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  sampler = Estimator()


Non-Clifford Ground state energy: -3.712497370567829
Using this function is only recommended for switching from ideal simulator to shot-noise or quantum hardware.
                 Multiple switching back and forth can lead to un-expected outcomes and is an experimental feature.

Reset RDMs, energies, and QI metrics.
Number of shots is None. Ideal simulator is assumed.
Your settings are:
 Ansatz:              tUPS
 Number of shots:     None
 ISA                  True
 Transpiled circuit   True
 Primitive:           Sampler
 Post-processing:     False
 Circuit layout:      [17, 25, 35, 24, 28, 26, 27, 44]
 Non-local gates:     405
 Transpilation strategy: External PassManager
 Primitive backend:   AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['cz', 'x', 'id', 'reset', 'measure', 'sx']>)

Noisy Non-Clifford Ground state energy: -2.9354609200029995


In [20]:
for i in range(len(optimized_parameters)):
    m = np.pi/2
    WF.ansatz_parameters[i] = m
    for n in range(9):
        if i == 1 + 3*n:
            WF.ansatz_parameters[i] = 4*m

print(WF.ansatz_parameters)

[1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966]


In [26]:
mol = pyscf.M(atom="""H 0.0 0.0 0.0;
                      H 1.5 0.0 0.0;
                      H 0.0 1.8 0.0;
                      H 1.5 1.8 0.0;""", basis="sto-3g", unit="angstrom")
rhf = pyscf.scf.RHF(mol).run()

sampler = Estimator()
primitive = sampler
mapper = JordanWignerMapper()
# For H4 you can make the wavefunction better by increasing n_layers.
# n_layers: 3 will prob. give almost the FCI solution.
QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 3, "do_pp": True}, ISA = True)

WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (4, 4),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    #include_active_kappa = True
)
WF.ansatz_parameters = [1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966]
#WF.ansatz_parameters = [np.pi/2 for _ in range(27)]
clifford_ground_state_energy = WF.energy_elec
print(clifford_ground_state_energy)

backend = FakeTorino()
QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
QI.do_postselection = False
QI.do_M_mitigation = False
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(backend_options={"noise_model":noise_model})
WF.change_primitive(sampler)
noisy_clifford_ground_state_energy = WF.energy_elec
print('exact:', clifford_ground_state_energy)
print('noisy:', noisy_clifford_ground_state_energy)

converged SCF energy = -1.77674731624872


Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3


/tmp/ipykernel_669176/3767076428.py:7: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  sampler = Estimator()


-3.2895813578825837
Using this function is only recommended for switching from ideal simulator to shot-noise or quantum hardware.
                 Multiple switching back and forth can lead to un-expected outcomes and is an experimental feature.

Reset RDMs, energies, and QI metrics.
Number of shots is None. Ideal simulator is assumed.
Your settings are:
 Ansatz:              tUPS
 Number of shots:     None
 ISA                  True
 Transpiled circuit   True
 Primitive:           Sampler
 Post-processing:     False
 Circuit layout:      [17, 25, 35, 24, 28, 26, 27, 44]
 Non-local gates:     405
 Transpilation strategy: External PassManager
 Primitive backend:   AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['cz', 'x', 'id', 'reset', 'measure', 'sx']>)

exact: -3.2895813578825837
noisy: -2.961345501868496


In [25]:
print(WF.ansatz_parameters)

[1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966, 1.5707963267948966]


In [7]:
mol = pyscf.M(atom="""H 0.0 0.0 0.0;
                      H 1.5 0.0 0.0;
                      H 0.0 1.8 0.0;
                      H 1.5 1.8 0.0;""", basis="sto-3g", unit="angstrom")
rhf = pyscf.scf.RHF(mol).run()

sampler = Estimator()
primitive = sampler
mapper = JordanWignerMapper()
# For H4 you can make the wavefunction better by increasing n_layers.
# n_layers: 3 will prob. give almost the FCI solution.
QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 4, "do_pp": True}, ISA = True)

WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (4, 4),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    include_active_kappa = True
)

#WF.run_vqe_2step("RotoSolve", maxiter=3)
#WF.run_vqe_2step("SLSQP", maxiter=40)
WF.ansatz_parameters = [3.1431228572975396, 2.7861493547159264, 3.1396692239259947, -3.142057339846848, 2.791150176990398, 3.144389766300438, -1.5653104599425836, 0.27240921676016555, 3.1384976277670154, 3.1210497755145403, 0.0655975291978254, 3.1386831001117295, 3.1672998707476467, 0.00045866400767861026, -3.113669666746792, -3.126391643474034, -0.034094707610928636, 3.151672138940099, 3.097362854855655, -0.0822620679689756, 3.0505562027648816, -3.0905175956936484, -0.011010502555054685, -3.0594652278673737, -3.149314869130154, -0.23511735777506634, 3.124078811244982, -3.400189577683653, -0.03766298071932297, -3.4773143551058996, 3.3796990528741597, 0.020211517238238847, 3.4558195393388034, 3.140869714110472, -0.08923065201729881, 3.1437217521757854]
optimized_parameters = WF.ansatz_parameters
#no noise
nonclif_ground_state_energy = WF.energy_elec
print("Non-Clifford Ground state energy:", nonclif_ground_state_energy)

#device noise
# Update the primitive with simulated noise
backend = FakeTorino()
QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
QI.do_postselection = False
QI.do_M_mitigation = False
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(backend_options={"noise_model":noise_model})
WF.change_primitive(sampler)    
# Calculate the ground state energy using the noisy simulator
noisy_nonclif_ground_state_energy = WF.energy_elec
print("Noisy Non-Clifford Ground state energy:",noisy_nonclif_ground_state_energy)

converged SCF energy = -1.77674731624872
Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3


/tmp/ipykernel_776274/2393043778.py:7: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  sampler = Estimator()


Non-Clifford Ground state energy: -3.712421442890217
Using this function is only recommended for switching from ideal simulator to shot-noise or quantum hardware.
                 Multiple switching back and forth can lead to un-expected outcomes and is an experimental feature.

Reset RDMs, energies, and QI metrics.
Number of shots is None. Ideal simulator is assumed.
Your settings are:
 Ansatz:              tUPS
 Number of shots:     None
 ISA                  True
 Transpiled circuit   True
 Primitive:           Sampler
 Post-processing:     False
 Circuit layout:      [44, 26, 27, 17, 43, 25, 35, 28]
 Non-local gates:     528
 Transpilation strategy: External PassManager
 Primitive backend:   AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['measure', 'id', 'cz', 'sx', 'x', 'reset']>)

Noisy Non-Clifford Ground state energy: -2.853327201865213


In [8]:
print(len(WF.ansatz_parameters))

36


In [9]:
for i in range(len(optimized_parameters)):
    m = np.pi/2
    WF.ansatz_parameters[i] = m
    for n in range(12):
        if i == 1 + 3*n:
            WF.ansatz_parameters[i] = 4*m

print(WF.ansatz_parameters)

[1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966]


In [14]:
mol = pyscf.M(atom="""H 0.0 0.0 0.0;
                      H 1.5 0.0 0.0;
                      H 0.0 1.8 0.0;
                      H 1.5 1.8 0.0;""", basis="sto-3g", unit="angstrom")
rhf = pyscf.scf.RHF(mol).run()

sampler = Estimator()
primitive = sampler
mapper = JordanWignerMapper()
# For H4 you can make the wavefunction better by increasing n_layers.
# n_layers: 3 will prob. give almost the FCI solution.
QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 4, "do_pp": True}, ISA = True)

WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (4, 4),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    include_active_kappa = True
)

#WF.run_vqe_2step("RotoSolve", maxiter=3)
#WF.run_vqe_2step("SLSQP", maxiter=40)
#WF.ansatz_parameters = [np.pi/2 for _ in range(36)]
WF.ansatz_parameters = [1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966, 1.5707963267948966, 6.283185307179586, 1.5707963267948966]
optimized_parameters = WF.ansatz_parameters
#no noise
nonclif_ground_state_energy = WF.energy_elec
print("Non-Clifford Ground state energy:", nonclif_ground_state_energy)

#device noise
# Update the primitive with simulated noise
backend = FakeTorino()
QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
QI.do_postselection = False
QI.do_M_mitigation = False
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(backend_options={"noise_model":noise_model})
WF.change_primitive(sampler)    
# Calculate the ground state energy using the noisy simulator
noisy_nonclif_ground_state_energy = WF.energy_elec
print("Noisy Non-Clifford Ground state energy:",noisy_nonclif_ground_state_energy)

converged SCF energy = -1.77674731624872
Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3


/tmp/ipykernel_776274/3126877671.py:7: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  sampler = Estimator()


Non-Clifford Ground state energy: -3.289581357882583
Using this function is only recommended for switching from ideal simulator to shot-noise or quantum hardware.
                 Multiple switching back and forth can lead to un-expected outcomes and is an experimental feature.

Reset RDMs, energies, and QI metrics.
Number of shots is None. Ideal simulator is assumed.
Your settings are:
 Ansatz:              tUPS
 Number of shots:     None
 ISA                  True
 Transpiled circuit   True
 Primitive:           Sampler
 Post-processing:     False
 Circuit layout:      [44, 26, 27, 17, 43, 25, 35, 28]
 Non-local gates:     528
 Transpilation strategy: External PassManager
 Primitive backend:   AerSimulator('aer_simulator'
             noise_model=<NoiseModel on ['measure', 'id', 'cz', 'sx', 'x', 'reset']>)

Noisy Non-Clifford Ground state energy: -2.9598300797108297
